In [1]:
import pandas as pd
import sqlite3

In [2]:
# connect to backend/databease.db
conn = sqlite3.connect('../database.db')
c = conn.cursor()

In [8]:
# baca excel .venv/db_peg.xlsx
df_user = pd.read_excel('../db_peg.xlsx', sheet_name='users', dtype=str)
df_user.columns 

Index(['id', 'username', 'password', 'role', 'nama', 'nip', 'email',
       'fotolink', 'grup'],
      dtype='object')

In [9]:
df_user.head()

,id,username,password,role,nama,nip,email,fotolink,grup
0,240002315,ib.wisnu@bpk.go.id,ib.wisnu@bpk.go.id,Fungsional Pemeriksa,Ida Bagus Ketut Wisnu,196711291994031002,ib.wisnu@bpk.go.id,https://sisdm.bpk.go.id/photo/240002315/md.jpg,Bidang Pemeriksaan Bali I
1,240002924,ida.sudiartha@bpk.go.id,ida.sudiartha@bpk.go.id,Fungsional Pemeriksa,Ida Bagus Made Sudiartha,196709281998031001,ida.sudiartha@bpk.go.id,https://sisdm.bpk.go.id/photo/240002924/md.jpg,Bidang Pemeriksaan Bali I
2,240002921,luh.parwati@bpk.go.id,luh.parwati@bpk.go.id,Fungsional Pemeriksa,Luh Made Putri Parwati,197503121998032001,luh.parwati@bpk.go.id,https://sisdm.bpk.go.id/photo/240002921/md.jpg,Bidang Pemeriksaan Bali I
3,240003436,iik.jayani@bpk.go.id,iik.jayani@bpk.go.id,Fungsional Pemeriksa,Desak Made Iik Jayani,197309052000032002,iik.jayani@bpk.go.id,https://sisdm.bpk.go.id/photo/240003436/md.jpg,Bidang Pemeriksaan Bali I
4,240002684,astiti.dewi@bpk.go.id,astiti.dewi@bpk.go.id,Fungsional Pemeriksa,Anak Agung Astiti Dewi,197110181997032001,astiti.dewi@bpk.go.id,https://sisdm.bpk.go.id/photo/240002684/md.jpg,Bidang Pemeriksaan Bali I


In [11]:
# membuat tabel users di database.db dengan kolom ['id', 'username', 'password', 'role', 'nama', 
# 'nip', 'email','fotolink', 'grup']

c.execute('''CREATE TABLE IF NOT EXISTS users
             (id TEXT PRIMARY KEY,
              username TEXT,
              password TEXT,
              role TEXT,
              nama TEXT,
              nip TEXT,
              email TEXT,
              fotolink TEXT,
              grup TEXT)''')
conn.commit()

In [12]:
# menambahkan data dari df_user ke tabel users dengan password di hash dengan sha256
import hashlib
for index, row in df_user.iterrows():
    password_hashed = hashlib.sha256(row['password'].encode()).hexdigest()
    c.execute('''INSERT INTO users (id, username, password, role, nama, nip, email, fotolink, grup)
                 VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)''',
              (row['id'], row['username'], password_hashed, row['role'], row['nama'], row['nip'], row['email'], row['fotolink'], row['grup']))
conn.commit()


In [13]:
# tambahkan data pegawai ke tabel pegawai di database.db
for index, row in df_peg.iterrows():
    c.execute("INSERT OR REPLACE INTO pegawai (id, nama, email, fotolink, jnsjab, jab, unker, subunker, statpeg, aktifpeg) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", 
              (row['id'], row['nama'], row['email'], row['fotolink'], row['jnsjab'], row['jab'], row['unker'], row['subunker'], row['statpeg'], row['aktifpeg']))
conn.commit()


In [15]:
df_st = pd.read_excel('../../.venv/db_rik.xlsx', sheet_name='st')
df_penugasan = pd.read_excel('../../.venv/db_rik.xlsx', sheet_name='tugas')

In [16]:
[df_st.columns,
df_penugasan.columns]

[Index(['id', 'thn', 'entabr', 'tglst', 'awalst', 'akhirst', 'stlink',
        'perihalst', 'jnsrik', 'tahaprik'],
       dtype='object'),
 Index(['id', 'idst', 'nama', 'peranabr', 'urutanst'], dtype='object')]

In [21]:
c.execute('''CREATE TABLE IF NOT EXISTS st
             (id TEXT PRIMARY KEY,
              thn INTEGER,
              entabr TEXT,
              tglst TEXT,
              awalst TEXT,
              akhirst TEXT,
              stlink TEXT,
              perihalst TEXT,
              jnsrik TEXT,
              tahaprik TEXT)''')

for index, row in df_st.iterrows():
    tglst = row['tglst'].strftime('%Y-%m-%d') if pd.notnull(row['tglst']) else None
    awalst = row['awalst'].strftime('%Y-%m-%d') if pd.notnull(row['awalst']) else None
    akhirst = row['akhirst'].strftime('%Y-%m-%d') if pd.notnull(row['akhirst']) else None
    c.execute(
        "INSERT OR REPLACE INTO st (id, thn, entabr, tglst, awalst, akhirst, stlink, perihalst, jnsrik, tahaprik) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
        (row['id'], row['thn'], row['entabr'], tglst, awalst, akhirst, row['stlink'], row['perihalst'], row['jnsrik'], row['tahaprik'])
    )
conn.commit()

In [22]:
# create tabel penugasan if not exist with colums 'id', 'idst', 'nama', 'peranabr', 'urutanst'
c.execute('''CREATE TABLE IF NOT EXISTS penugasan
             (id TEXT PRIMARY KEY,
              idst TEXT,
              nama TEXT,
              peranabr TEXT,
              urutanst INTEGER)''')

for index, row in df_penugasan.iterrows():
    c.execute(
        "INSERT OR REPLACE INTO penugasan (id, idst, nama, peranabr, urutanst) VALUES (?, ?, ?, ?, ?)",
        (row['id'], row['idst'], row['nama'], row['peranabr'], row['urutanst'])
    )   
conn.commit()